In [2]:
import requests
import pandas as pd
# import matplotlib
import os
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from secrets import ALPACA_API_KEY_VAR

In [3]:
# Set some constant variables, I could put all of this in a seperate config file
ALPACA_API_KEY = ALPACA_API_KEY_VAR
START_DATE = '2005-01-03'
END_DATE = '2020-12-31'
# URL for all the tickers on Polygon
POLYGON_TICKERS_URL = 'https://api.polygon.io/v2/reference/tickers?page={}&apiKey={}'
# URL FOR PRICING DATA - Note, getting pricing that is UNADJUSTED for splits, I will try and adjust those manually
POLYGON_AGGS_URL = 'https://api.polygon.io/v2/aggs/ticker/{}/range/1/day/{}/{}?unadjusted=true&apiKey={}'
# URL FOR DIVIDEND DATA
POLYGON_DIV_URL = 'https://api.polygon.io/v2/reference/dividends/{}?apiKey={}'
# URL FOR STOCK SPLITS
POLYGON_SPLIT_URL = 'https://api.polygon.io/v2/reference/splits/{}?apiKey={}'
#URL FOR TICKER TYPES
POLYGON_TYPES_URL = 'https://api.polygon.io/v2/reference/types?apiKey={}'
#URL FOR TICKER Info
POLYGON_TICKER_INFO_URL = "https://api.polygon.io/v1/meta/symbols/{}/company?apiKey={}"

## Ticker Info API

In [3]:
requests.get(POLYGON_TICKER_INFO_URL.format("AAPL", ALPACA_API_KEY)).text

'{"logo":"https://s3.polygon.io/logos/aapl/logo.png","listdate":"1990-01-02","cik":"320193","bloomberg":"EQ0010169500001000","figi":null,"lei":"HWUPKR0MPOU8FGXBT394","sic":3571,"country":"usa","industry":"Computer Hardware","sector":"Technology","marketcap":908316631180,"employees":123000,"phone":"+1 408 996-1010","ceo":"Timothy D. Cook","url":"http://www.apple.com","description":"Apple Inc is designs, manufactures and markets mobile communication and media devices and personal computers, and sells a variety of related software, services, accessories, networking solutions and third-party digital content and applications.","exchange":"Nasdaq Global Select","name":"Apple Inc.","symbol":"AAPL","exchangeSymbol":"NGS","hq_address":"1 Infinite Loop Cupertino CA, 95014","hq_state":"CA","hq_country":"USA","type":"CS","updated":"11/16/2018","tags":["Technology","Consumer Electronics","Computer Hardware"],"similar":["MSFT","NOK","IBM","HPQ","GOOGL","BB","XLK"],"active":true}'

In [4]:
# Get the list of all supported tickers from Polygon.io
def get_tickers(url = POLYGON_TICKERS_URL):
    page = 1

    session = requests.Session()
    # Initial request to get the ticker count
    r = session.get(POLYGON_TICKERS_URL.format(page, ALPACA_API_KEY))
    data = r.json()

    # This is to figure out how many pages to run pagination 
    count = data['count']
    print('total tickers ' + str(count))
    pages = math.ceil(count / data['perPage'])

    # Pull in all the pages of tickers
    for pages in range (2, pages+1):  # For production
    # for pages in range (2, 10):  # For testing
        r = session.get(POLYGON_TICKERS_URL.format(page, ALPACA_API_KEY))
        data = r.json()
        df = pd.DataFrame(data['tickers'])
        df.to_csv('data/tickers/{}.csv'.format(page), index=False)
        print('Page {} processed'.format(page))
        page += 1
        
    return('Processes {} pages of tickers'.format(page-1))


# Stich all of these csv files into one dataframe for analysis
def combine_tickers(directory):

    df = pd.DataFrame()

    for f in os.listdir(directory):
        df2 = pd.read_csv('{}/{}'.format(directory, f))
        df = df.append(df2)
    
    # Read out a copy of the file to a csv for later analysis
    df.set_index('ticker', inplace=True)
    df.drop_duplicates()  # Just in case any tickers get pulled twice
    df.to_csv('polygon_tickers.csv')
        
    return df


def filter_us_exch(ticker_df):
    
    # Keep only U.S. Dollar denominated securities
    df = ticker_df[(ticker_df.currency == 'USD') & (ticker_df.locale == 'US')]
    # Keep only the primary U.S. exchanges
    exch = ['AMX','ARCA','BATS','NASDAQ','NSC','NYE']
    df = df[df['primaryExch'].isin(exch)]
    # Filter out preferred stock, american depositry receipts, closed end funds, reit
    stockTypes = ['PFD','ADR','CEF','MLP','REIT','RIGHT','UNIT','WRT']
    df = df[df['type'].isin(stockTypes) == False]
    
    df.to_csv('polygon_tickers_us.csv')

    # Create a list of symbols to loop thru
    symbols = df.index.tolist()

    return symbols


# Get the aggregated bars for the symbols I need
def get_bars(symbolslist, outdir, start, end):

    session = requests.Session()
    # In case I run into issues, retry my connection
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))
    count = 0
    
    barlog = open("barlog.txt", "w")
    
    for symbol in symbolslist:
        try:
            r = session.get(POLYGON_AGGS_URL.format(symbol, start, end, ALPACA_API_KEY))
            if r:
                data = r.json()
            
                # create a pandas dataframe from the information
                if data['queryCount'] > 1:
                    df = pd.DataFrame(data['results'])
                    df['date'] = pd.to_datetime(df['t'], unit='ms')
                    df['date'] =  df['date'].dt.date.astype(str)
                    df.set_index('date', inplace=True)
                    df['symbol'] = symbol

                    df.drop(columns=['vw', 't', 'n'], inplace=True)
                    df.rename(columns={'v': 'volume', 'o': 'open', 'c': 'close', 'h': 'high', 'l': 'low'}, inplace=True)

                    df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
                    count += 1

                    # Logging, I could write a short method for this to reuse
                    msg = (symbol + ' file created with record count ' + str(data['queryCount']))
                    print(msg)
                    barlog.write(msg)
                    barlog.write("\n")

                else:
                    msg = ('No data for symbol ' + str(symbol))
                    print(msg)
                    barlog.write(msg)
                    barlog.write("\n")
            else:
                msg = ('No response for symbol ' + str(symbol))
                print(msg)
                barlog.write(msg)
                barlog.write("\n")
        # Raise exception but continue           
        except:
            msg = ('****** exception raised for symbol ' + str(symbol))
            print(msg)
            barlog.write(msg)
            barlog.write("\n")
    
    barlog.close()
    return ('{} file were exported'.format(count))


# Define a function to pull in the splits data
def get_splits(symbolslist, outdir):

    session = requests.Session()
    # In case I run into issues, retry my connection
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])

    session.mount('http://', HTTPAdapter(max_retries=retries))
    count = 0
    
    # Get the split data
    for symbol in symbolslist:
        try:
            r = session.get(POLYGON_SPLIT_URL.format(symbol, ALPACA_API_KEY))
            if r:
                data = r.json()
                if data['count'] > 0:
                    df = pd.DataFrame(data['results'])
                    df.rename(columns={'exDate': 'date', 'declaredDate': 'splitDeclaredDate'}, inplace=True)
                    df.drop(columns=['paymentDate'], inplace=True)
                    df.set_index('date', inplace=True)
                    df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
                    
                    print('split file for ' + symbol + ' ' + str(data['count']))
                    count += 1
                else:
                    print('No data for symbol ' + str(symbol))
            else:
                print('No response for symbol ' + str(symbol))
        # Raise exception but continue           
        except:
            print('****** exception raised for symbol ' + str(symbol))
            
    return ('{} file were exported'.format(count))


# Fix erroneous splits from a correction file manually created
def fix_splits(splitpath):
    # Get the split corrections to overwrite
    correct_df = pd.read_csv('split_corrections.csv')
    # create a list of symbols to fix
    symbols = correct_df['ticker'].tolist()
    # remove duplicates
    symbols = list(dict.fromkeys(symbols))

    # for symbol in symbols:
    for symbol in symbols:
        print(symbol)

    # get any splits
        if os.path.isfile('{}/{}.csv'.format(splitpath, symbol)):
            df = pd.read_csv('{}/{}.csv'.format(splitpath, symbol))
            print(df)
            df = pd.merge(df, correct_df, how='left', left_on=['date', 'ticker'], right_on=['date', 'ticker'])
            
            for index, row in df.iterrows():
                # Adjust bad dates
                if not pd.isnull(row.date_adj):
                    df.loc[index, 'date'] = row.date_adj
                # Adjust bad ratios
                if not pd.isnull(row.ratio_adj):
                    df.loc[index, 'ratio'] = row.ratio_adj
                else:
                    df.loc[index, 'ratio'] = row.ratio_x
            
            # Format the dataframe for export
            df = df[['date', 'ticker', 'ratio']]
            df.set_index('date', inplace=True)
            print(df)

            # Overwrite the file with this new file
            df.to_csv('{}/{}.csv'.format(splitpath, symbol))
            print('Split file for {} corrected'.format(symbol))
            
        else:
            print('no file found')
                
    return ('Split file corrections complete')


# Define a function to pull in the splits data
def get_divs(symbolslist, outdir):

    session = requests.Session()
    count = 0
    
    # Get the split data
    for symbol in symbolslist: # ['AAPL']:
        r = session.get(POLYGON_DIV_URL.format(symbol, ALPACA_API_KEY))
        data = r.json()
        if data['count'] > 0:
            df = pd.DataFrame(data['results'])
            # df.rename(columns={'paymentDate': 'date'}, inplace=True)
            df.rename(columns={'exDate': 'date', 'amount': 'dividend',
                               'paymentDate': 'divPaymentDate',
                               'recordDate': 'divRecordDate',
                               'declaredDate': 'divDeclaredDate'}, inplace=True)
            df.set_index('date', inplace=True)
            df = df.groupby(df.index).first()
            df.to_csv('{}/{}.csv'.format(outdir, symbol), index=True)
            
            print('div file for ' + symbol + ' ' + str(data['count']))
            count += 1
            
    return ('{} file were exported'.format(count))


# Combine bars, splits and dividend
def combine_bars(barpath, splitpath, divpath):

    count = 0
    for f in os.listdir(barpath):
        
        symbol = f[:-4]
        print(symbol)
        
        # Get the bar data
        if os.path.isfile('{}/{}.csv'.format(barpath, symbol)):
            bars = pd.read_csv('{}/{}.csv'.format(barpath, symbol), index_col='date')
            
            # get any splits
            if os.path.isfile('{}/{}.csv'.format(splitpath, symbol)):
                splits = pd.read_csv('{}/{}.csv'.format(splitpath, symbol), index_col='date')
                splits.drop(columns=['ticker'], inplace=True)
                
                bars = bars.merge(splits, left_index=True, right_index=True, how='left')

            else:
                
                bars = bars
            
            # get any dividend payments
            if os.path.isfile('{}/{}.csv'.format(divpath, symbol)):
                divs = pd.read_csv('{}/{}.csv'.format(divpath, symbol), index_col='date')
                divs.drop(columns=['ticker'], inplace=True)
            
                bars = bars.merge(divs, left_index=True, right_index=True, how='left')
            
            else:
                
                bars = bars
                
            # Export bars 
            bars.to_csv('data/bars_adj/{}.csv'.format(symbol))
            count += 1
        
    return ('{} adjusted bar file were exported'.format(count))


# Adjust the OHLCV data for stock splits
def adj_bars(directory):

    count = 0
    for f in os.listdir(directory):

        df = pd.read_csv('{}/{}'.format(directory, f), index_col='date')
        
        if 'ratio' in df.columns:
            df['ratio_adj'] = df['ratio']
        else:
             df['ratio_adj'] = 1

        # Create a split factor, shifted to the day earlier.  Also, fill in any missing factors with 1
        df['split_factor'] = (1 / df['ratio_adj'].shift(-1)).fillna(1)
        #  Create a cumulative product of the splits, in reverse order using the []::-1]
        df['split_factor'] = df['split_factor'][::-1].cumprod()

        # Adjust the various OHLCV metrics
        df['volume_adj'] = df['volume'] * df['split_factor']
        df['open_adj'] = df['open'] / df['split_factor']
        df['close_adj'] = df['close'] / df['split_factor']
        df['high_adj'] = df['high'] / df['split_factor']
        df['low_adj'] = df['low'] / df['split_factor']
        df['dollar_volume'] = df['volume'] * df['close']

        df.to_csv('{}/{}'.format(directory, f))
        count += 1
        
    return ('{} files was adjusted'.format(count))

In [5]:
sp_constituents = pd.read_json("https://datahub.io/core/s-and-p-500-companies/r/constituents.json")
sp_constituents.head(3)

,Name,Sector,Symbol
0,3M Company,Industrials,MMM
1,A.O. Smith Corp,Industrials,AOS
2,Abbott Laboratories,Health Care,ABT


In [6]:
#%%  Filter down to the tickers I'm interestead in (this could also be done by modifying get_tickers)
# symbols = filter_us_exch(symbols)

symbols = list(sp_constituents.iloc[:, -1])

#%% Get all the aggregated bar/pricing data for each symbol in the filtered list
get_bars(symbols, 'data/bars', START_DATE, END_DATE)

reated with record count 4008
ADBE file created with record count 4028
AAP file created with record count 4028
AMD file created with record count 4028
AES file created with record count 4028
AFL file created with record count 4028
A file created with record count 4028
APD file created with record count 4028
AKAM file created with record count 4028
ALK file created with record count 4028
ALB file created with record count 4028
ARE file created with record count 4028
ALXN file created with record count 4028
ALGN file created with record count 4028
ALLE file created with record count 1784
LNT file created with record count 4028
ALL file created with record count 4028
GOOGL file created with record count 1700
GOOG file created with record count 4028
MO file created with record count 4028
AMZN file created with record count 4028
AMCR file created with record count 1010
AEE file created with record count 4028
AAL file created with record count 1779
AEP file created with record count 4028
AXP

'505 file were exported'

In [7]:
#%%  Pull in all the stock splits
get_splits(symbols, 'data/splits')

split file for MMM 1
split file for AOS 4
split file for ABT 3
No data for symbol ABBV
split file for ABMD 1
No data for symbol ACN
split file for ATVI 6
split file for ADBE 3
split file for AAP 2
split file for AMD 1
split file for AES 1
split file for AFL 4
No data for symbol A
split file for APD 1
No data for symbol AKAM
split file for ALK 2
split file for ALB 1
split file for ARE 1
split file for ALXN 2
No data for symbol ALGN
No data for symbol ALLE
split file for LNT 1
split file for ALL 1
split file for GOOGL 1
split file for GOOG 2
No data for symbol MO
split file for AMZN 3
No data for symbol AMCR
No data for symbol AEE
No data for symbol AAL
No data for symbol AEP
split file for AXP 2
split file for AIG 4
No data for symbol AMT
No data for symbol AWK
No data for symbol AMP
split file for ABC 3
split file for AME 4
split file for AMGN 2
split file for APH 4
split file for ADI 1
split file for ANSS 2
split file for ANTM 1
split file for AON 1
split file for APA 3
split file for

'332 file were exported'

In [8]:
# Fix data for about 50 splits from a correction file created manually
fix_splits('data/splits')

#%%  Pull in all the dividend data
get_divs(symbols, 'data/divs')

#%%  Combine the bars (pricing data) with any splits and dividend payments
combine_bars('data/bars', 'data/splits', 'data/divs')

#%%  Create new and stock split adjusted OHLCV fields
adj_bars('data/bars_adj')

#%%
# bars = pd.read_csv('data/bars_adj/AAPL.csv')
# bars['close'].plot()

MA
         date ticker  ratio  tofactor  forfactor
0  2014-01-21     MA    0.1         1         10
           ticker  ratio
date                    
2014-01-22     MA    0.1
Split file for MA corrected
NFLX
         date ticker splitDeclaredDate     ratio  tofactor  forfactor
0  2015-07-14   NFLX        2015-06-23  0.142857       1.0        7.0
1  2004-02-12   NFLX               NaN  0.500000       NaN        NaN
           ticker  ratio
date                    
2015-07-15   NFLX  0.143
2004-02-12   NFLX  0.500
Split file for NFLX corrected
BRK.A
no file found
EWZ
no file found
JEF
no file found
WY
         date ticker  ratio
0  2010-07-19     WY    0.5
           ticker  ratio
date                    
2010-07-20     WY    0.5
Split file for WY corrected
STN
no file found
CTSH
         date ticker splitDeclaredDate     ratio  tofactor  forfactor
0  2014-03-06   CTSH        2014-02-04  0.500000       1.0        2.0
1  2007-10-17   CTSH               NaN  0.500000       NaN        NaN


'505 files was adjusted'

# Financial Data API

In [37]:
POLYGON_FINANCIALS_URL = f"https://api.polygon.io/v2/reference/financials/JWN?&type=Q&sort=calendarDate&apiKey={ALPACA_API_KEY}"

In [38]:
session = requests.Session()

r = session.get(POLYGON_FINANCIALS_URL)

pd.DataFrame(r.json())

,status,results
0,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
1,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
2,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
3,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
4,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
...,...,...
70,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
71,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
72,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."
73,OK,"{'ticker': 'JWN', 'period': 'Q', 'calendarDate..."


In [39]:
final_data = []

for i in r.json()["results"]:
    x = pd.DataFrame(columns=[x[0] for x in list(i.items())],data=i, index=[0])
    final_data.append(x)
pd.concat(final_data).reset_index(drop=True)

,ticker,period,calendarDate,reportPeriod,updated,dateKey,accumulatedOtherComprehensiveIncome,assets,assetsCurrent,assetsNonCurrent,...,netCashFlowFromOperations,effectOfExchangeRateChangesOnCash,shareBasedCompensation,enterpriseValueOverEBIT,enterpriseValueOverEBITDA,priceEarnings,priceToEarningsRatio,priceSales,priceToSalesRatio,weightedAverageSharesDiluted
0,JWN,Q,2001-06-30,2001-07-31,2020-03-20,2001-09-07,1118000,3792151000,1911644000,1880507000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JWN,Q,2001-09-30,2001-10-31,2020-03-20,2001-12-07,1789000,3975151000,2021878000,1953273000,...,-63501000.0,0.0,-889000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JWN,Q,2001-12-31,2002-01-31,2020-03-20,2002-04-18,649000,4048779000,2054598000,1994181000,...,253489000.0,0.0,1177000.0,16.0,8.936,26.828,26.720,0.594,0.594,NaN
3,JWN,Q,2002-03-31,2002-04-30,2020-03-20,2002-06-07,-3538000,4024430000,2015154000,2009276000,...,30591000.0,0.0,1131000.0,19.0,NaN,43.768,42.842,0.583,0.583,NaN
4,JWN,Q,2002-06-30,2002-07-31,2020-03-20,2002-09-12,-714000,4231513000,2188368000,2043145000,...,182683000.0,0.0,956000.0,17.0,8.604,38.740,38.055,0.490,0.490,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,JWN,Q,2018-12-31,2019-02-02,2020-03-20,2019-03-18,-37000000,7886000000,3374000000,4512000000,...,654000000.0,0.0,18000000.0,10.0,5.696,12.145,13.113,0.432,0.466,167000000.0
71,JWN,Q,2019-03-31,2019-05-04,2020-03-20,2019-06-05,-46000000,9338000000,2958000000,6380000000,...,-31000000.0,0.0,20000000.0,12.0,6.563,9.616,10.310,0.314,0.315,156200000.0
72,JWN,Q,2019-06-30,2019-08-03,2020-03-20,2019-09-04,-39000000,9935000000,3483000000,6452000000,...,723000000.0,0.0,20000000.0,12.0,5.720,9.273,9.707,0.294,0.294,155600000.0
73,JWN,Q,2019-09-30,2019-11-02,2020-03-20,2019-12-04,-38000000,10075000000,3573000000,6502000000,...,-123000000.0,0.0,15000000.0,12.0,6.343,10.598,10.922,0.378,0.378,155800000.0


In [35]:
pd.DataFrame(columns=[x[0] for x in list(i.items())],data=i)

ValueError: If using all scalar values, you must pass an index

In [36]:
x

,ticker,period,calendarDate,reportPeriod,updated,dateKey,accumulatedOtherComprehensiveIncome,assets,assetsCurrent,assetsNonCurrent,...,shares,weightedAverageShares,weightedAverageSharesDiluted,salesPerShare,tangibleAssetValue,taxAssets,incomeTaxExpense,taxLiabilities,tangibleAssetsBookValuePerShare,workingCapital
0,MSFT,Q,2020-03-31,2020-03-31,2020-04-29,2020-04-29,2676000000,285449000000,170505000000,114944000000,...,7583440247,7602000000,7675000000,4.607,236530000000,0,2091000000,32369000000,31.114,111798000000


In [14]:
len([x[1] for x in list(i.items())])

49

In [15]:
len([x[0] for x in list(i.items())])

49

In [38]:
i.values()

dict_values(['AAPL', 'QA', '2019-03-31', '2019-03-30', '2020-05-01', '2019-03-30', -1499000000, 341998000000, 123346000000, 218652000000, 22.648, -2363000000, 37988000000, 37988000000, 36194000000, 11561000000, 1.315, 2.231, 112630000000, 22429000000, 90201000000, 112630000000, 5532000000, 3040000000, 0, 0.015, 0.73, 13793000000, 16833000000, 0.29, 16833000000, 13793000000, 13793000000, 2.47, 2.46, 2.47, 105860000000, 105860000000, 965626436000, 14, 12.183, 8792000000, 1.881, 1, 21821000000, 0.376, 0, 0, 322868000000, 4884000000, 187423000000, 42104000000, 145319000000, 236138000000, 93772000000, 142366000000, 895667436000, -4954000000, -124000000, -23312000000, -2542000000, -3443000000, -29457000000, 13348000000, 15749000000, 11155000000, 0, 11561000000, 11561000000, 11561000000, 0, 0, 0.199, 8406000000, 13415000000, 30443000000, 0.296, 8.461, 15.667, 15.842, 38746000000, 0, 189.95, 3.465, 3.435, 26278000000, 64558000000, 58015000000, 58015000000, 3948000000, 1514000000, 4458000000, 1